# Elastix

In this notebook, we will learn the basics of [elastix](https://elastix.lumc.nl/), the image registration software we will use in this course. You can find the documentation for SimpleElastix (the python wrapper for elastix), [here](https://simpleelastix.readthedocs.io/index.html).


First read through the function `run_elastix` below. It mirrors closely [this example from the SimpleElastix documentation](https://simpleelastix.readthedocs.io/HelloWorld.html#object-oriented-interface).  It is not necessary to understand the details.

In this notebook, we will begin to look at some of the different parameters we can give to elastix, and their effects.  This will be the focus of the course. You can read more about the parameter maps [here](https://simpleelastix.readthedocs.io/ParameterMaps.html).

In [ ]:
import SimpleITK as sitk

import itkwidgets
from itkwidgets import view

def run_elastix(fixed_image, moving_image, params):
    """
    Runs elastix with the given fixed and moving images,
    and elastix parameters.
    
    Returns the ElastixImageFilter, from which the results
    can be obtained.
    """
    
    # Make the image filter 
    elastixImageFilter = sitk.ElastixImageFilter()
    
    # Set the moving and fixed images
    elastixImageFilter.SetFixedImage(fixed_image)
    elastixImageFilter.SetMovingImage(moving_image)
    
    # Set the parameters
    elastixImageFilter.SetParameterMap(params)
    
    # Run it
    elastixImageFilter.Execute()
    
    # Return the filter
    # It's posible to get the results from the filter
    return elastixImageFilter

##  Goal

In this notebook, we will use an example computed tomography (CT) image.  I've artificially translated it by a large amount, and saved the result in the file `ct_translated.tif`. The goal of this notebook is to find that translation, using image registration.

First let's load the images. The function `sitk.ReadImage` loads an imaage from a file.

In [ ]:
fixedImage = sitk.ReadImage('../sampleImages/ct_2d_moving.tif')
movingImage= sitk.ReadImage('../sampleImages/ct_translated.tif')

Let's visualize the images with another helpful library: [itkwidgets](https://itkwidgets.readthedocs.io/en/latest/)

The function `itkwidgets.compare` displays two images side-by-side. Run the cell below.  

Note: click the triple bar (or "hamburger" icon in the top left of each window to minimize the settings panel.

In [ ]:
itkwidgets.compare(movingImage,fixedImage)

Mouse over the images above, looking at the "Position" displayed at the bottom. Pick a point that you can find in both images, and make note of the positions. 

* What do you think the correct translation is?

Elastix has default sets of parameters for different transformation types that we can get with the function `sitk.GetDefaultParameterMap`. In this example we will use the `'translation'` transformation type.

In [ ]:
params = sitk.GetDefaultParameterMap('translation')
sitk.PrintParameterMap(params)

There are lots of options, but focus on these:

* `(Transform "TranslationTransform")`
    * elastix will find a translation
* `(NumberOfResolutions 4.000000)`
    * there are four resolution levels
* `(MaximumNumberOfIterations 256.000000)`
    * elastix will run a maximum of 256 iterations at each resolution level
* `(AutomaticTransformInitialization "true")`
    * After you run elastix, look for the text `Transform parameters are initialized as:` to see what happened
    * read about how this works [here](https://elastix.lumc.nl/doxygen/classelastix_1_1TranslationTransformElastix.html)
    
Now let's run elastix using these parameters with the functino `run_elastix` we defined above.

Then we get the transformed moving image with `elastixImageFilter.GetResultImage()` and visualize the results again with `itkwidgets.compare`

In [ ]:
elastixImageFilter = run_elastix(fixedImage, movingImage, params)
transformedMovingImage = elastixImageFilter.GetResultImage()

In [ ]:
itkwidgets.compare(transformedMovingImage,fixedImage)

Notice that the images are not well registered.  The moving image moved in the right direction, but did not go far enough. Sometimes, this is because the algorithm stops too early - i.e. did not run for enough iterations.  Let's increase the number of iterations to see if that helps.

We'll set the number of iterations to `2000`, which is about 8x more than our previous run.

Note: for technical reasons we have to to set the number of iterations with a list containing a string like this `['2000']`, but details are not important.

After running the cell below, check that the `MaximumNumberOfIterations` are `2000` as we want.

In [ ]:
params_more_iterations = sitk.GetDefaultParameterMap('translation')
params_more_iterations['MaximumNumberOfIterations'] = ['2000']
sitk.PrintParameterMap(params_more_iterations)

In [ ]:
elastix_more_iterations = run_elastix(fixedImage, movingImage, params_more_iterations)
result_more_iterations = elastix_more_iterations.GetResultImage()

In [ ]:
itkwidgets.compare(result_more_iterations,fixedImage)

Observe these things:

* It took longer for the algorithm to run
* The resulting transformed image looks the same as before (adding more iterations didn't help)

If you look through the output, you will some lines that look approximately like this:

```
1997	-0.035259	152.702935	9.012013	0.003789	0.6
1998	-0.042390	153.702593	8.960445	0.002874	0.6
1999	-0.043482	154.686703	8.910253	0.003916	0.6
Time spent in resolution 1 (ITK initialization and iterating): 1.135 s.
Stopping condition: Maximum number of iterations has been reached.
```

The left-most number is the iteration count, and indicates that elastix did indeed run for 2000 iterations, and stopped because it reached the maximum number of iterations. 

Since that didn't help, let's try loading elastix parameters from a file using the function `sitk.ReadParameterFile`, and print them with `sitk.PrintParameterMap`:

In [ ]:
params_affineMI = sitk.ReadParameterFile('../elastixParameters/AffineMI_2d.txt')
sitk.PrintParameterMap( params_affineMI )

Notice this part of the parameters: `(Transform "AffineTransform")`, telling elastix to find an affine, and not just a translation.  Will this help?  Check the values of the other important parameters we've looked at:

* `Transform`
* `NumberOfResolutions`
* `MaximumNumberOfIterations`
* `AutomaticTransformInitialization`
    * if not present defaults to "false"
    
Even though we don't have much intuition about these parameters yet, **make a prediction about whether this will perform better, worse, or the same compared to the previous**. Making and correctiong predictions is very helpful during learning!

When you have a guess, run the two cells below to see what happens.

In [ ]:
elastixImageFilter = run_elastix(fixedImage, movingImage, params_affineMI)
result_affine_mi = elastixImageFilter.GetResultImage()

In [ ]:
itkwidgets.compare(result_affine_mi,fixedImage)

What happened? [Click here](https://github.com/bogovicj/emboBioImage2023_registration/wiki/Spoiler-result-1) for an observation and explanation for the above result.

Now we'll find a set of parameters that gives a good result.

In [ ]:
params_good = sitk.GetDefaultParameterMap('translation')
params_good['AutomaticTransformInitialization'] = ['true']
params_good['AutomaticTransformInitializationMethod'] = ['CenterOfGravity']
params_good['MaximumNumberOfIterations'] = ['128']
sitk.PrintParameterMap(params_good)

In [ ]:
elastixImageFilter = run_elastix(fixedImage, movingImage, params_good)
result_good = elastixImageFilter.GetResultImage()

In [ ]:
itkwidgets.compare(result_good,fixedImage)

The lines below will print the translation of the moving image to match the fixed image, it will look like:

```
(tx, ty)
```

where `tx` is the translation in `x` (horizontal) and where `ty` is the translation in `y` (vertical).
Note that the positive `y` direction points down.


**Predict what the translation will be (Hint: remember the video lecture)**'

After you've made a prediction, run the cell below to see the result.

In [ ]:
transformation = elastixImageFilter.GetTransformParameterMap()
transformation[0]['TransformParameters']

**Does the output match your prediction?**

Explain the result, now that you see it (hint: remember the video lecture).


After you've thought about it, [click here to check your explanation](https://github.com/bogovicj/emboBioImage2023_registration/wiki/Spoiler-2).